In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [7]:

def create_np_arry_from_ASC_file (num_header, file_name):
    """This function creates from an .asc file an numpy array exlcuding the lines of the header 

    Args:
        num_header ([int]): [number of lines from .asc file that are not integrated in the numpy array]
        file_name ([str]): [Path to the .asc file]

    Returns:
        [numpy array]: [An Array with all the elements from the asc file exluding the header ]
    """
    x = num_header
    ### Creating Numpy Array of 
    with open(file_name, 'r') as handle:
        text = handle.readlines()

    processed_txt = []   
    for line in text[num_header:]: #skip the header
        element= line.strip(" ")
        element= element.replace("\n", "")
        element= element.split(" ")
        processed_txt.append(element)

    np_raw = np.array(processed_txt)
    np_raw =np_raw.astype(float)
    return np_raw

def ACC (num_both_snow, num_both_NOT_snow, num_sample):
    acc = (num_both_snow + num_both_NOT_snow) / num_sample
    return acc

def BIAS (num_snow_observed, num_snow_modeled):
    bias = num_snow_modeled/ num_snow_observed
    return bias

def CSI(num_both_snow, num_sample, num_both_NOT_snow):
    csi = (num_both_snow / (num_sample - num_both_NOT_snow))
    return csi

def valided_area_snow_modeld(validation_date, np_observe, np_model, snow_treshold):
    print(f"\nValidating at spatial scale of the  snow_cover from {validation_date} ")
    if np_observe.shape != np_model.shape:
        print(f"the shape of validation array is {np_observe.shape} and of the model array it is {np_model.shape}")
        raise ValueError("Both input arrays need to have the same size of col & rows -> return to the input files!")
    
    observed_snow = 0
    modeld_snow = 0
    both_snow = 0
    both_NOT_snow = 0
    unqueal = 0
    no_value = 0

    for i in range(np_model.shape[0]):
        for j in range(np_model.shape[1]):
            if np.isnan(np_model[i,j]) == True or np_observe[i,j] == 205:
                no_value  += 1
                continue
            elif np_model[i,j] > snow_treshold and np_observe[i,j] == 100: #is snow
                both_snow +=1
            elif np_model[i,j] <= snow_treshold and np_observe[i,j] == 0:
                both_NOT_snow += 1
            else:
                unqueal += 1
                
            if np_model[i,j] >= snow_treshold:
                modeld_snow += 1
            if np_observe[i,j] == 100:
                observed_snow += 1
    sample = np_model.shape[0]*np_model.shape[1] - no_value
    acc = ACC(both_snow, both_NOT_snow, sample)
    bias = BIAS(observed_snow, modeld_snow)
    csi = CSI(both_snow, sample, both_NOT_snow)
    print("Accuracy: ", acc, "Bias: ", bias, "Critical success Index: ", csi)
    return (acc, bias, csi)
    
def valided_area_snow_modeld_return_asc_file(validation_date, np_observe, np_model, snow_treshold, header):
    print(f"Creating a .asc file showing the validation of at spatial scale of the  snow_cover from {validation_date} ")
    if np_observe.shape != np_model.shape:
        print(f"the shape of validation array is {np_observe.shape} and of the model array it is {np_model.shape}")
        raise ValueError("Both input arrays need to have the same size of col & rows -> return to the input files!")
    
    snow_treshold
    modeld_snow_TRUE = 1
    modeld_NO_snow_TRUE = 0 
    modeld_snow_FALSE = -1
    modeld_NO_snow_FALSE = -5

    validation_array = np.full(np_model.shape, np.nan)

    for i in range(np_model.shape[0]):
        for j in range(np_model.shape[1]):
            if np.isnan(np_model[i,j]) == True or np_observe[i,j] == 205:
                validation_array[i,j] = 999
            elif np_model[i,j] > snow_treshold and np_observe[i,j] == 100: #is snow
                validation_array[i,j] = modeld_snow_TRUE
            elif np_model[i,j] <= snow_treshold and np_observe[i,j] == 0:
                validation_array[i,j] =modeld_NO_snow_TRUE
            elif np_model[i,j] > snow_treshold and np_observe[i,j] == 0:
                validation_array[i,j] = modeld_snow_FALSE
            else:
                validation_array[i,j] = modeld_NO_snow_FALSE
    np.savetxt(f"Diff_Illustrating_{validation_date}.asc", validation_array, header=header, delimiter=" ", fmt="%1.0f")
    with open(f"Diff_Illustrating_{validation_date}.asc", 'r') as handle:
        text = handle.readlines()
        for i in range(len(text)-1):
            if text[i] == "# \n":
                del text[i]
            if text[i][0] == "#":
                text[i] = text[i][2:]
            else:
                text[i] = f" {text[i]}"   
    with open(f"Diff_Illustrating_{validation_date}.asc", 'w') as f:
        for i in text:
            f.write(i)


In [8]:
def validation_area_import_keyfigures_asc_file (a_date, snow_treshold):
    input_file_observ = f"results/{a_date}.asc"
    input_file_model = f"results/{a_date}_MODEL.asc"
    input_file_model_SCF12 = f"results/{a_date}_MODEL-SCF1,2.asc"
    input_file_model_SCF14 = f"results/{a_date}_MODEL-SCF1,4.asc"

    with open(input_file_observ, "r") as header:
        a = header.readlines()[:5]
        head = ""
        head_rofental_asc_files = head.join(a)

    observe1_raw = create_np_arry_from_ASC_file(5, input_file_observ)
    model1_raw = create_np_arry_from_ASC_file(5, input_file_model)
    model1_SCF12_raw = create_np_arry_from_ASC_file(5, input_file_model_SCF12)
    model1_SCF14_raw = create_np_arry_from_ASC_file(5, input_file_model_SCF14)

    valided_area_snow_modeld(f"{a_date} (without SCF)", observe1_raw, model1_raw, snow_treshold)
    valided_area_snow_modeld(f"{a_date} (with SCF1,2)", observe1_raw, model1_SCF12_raw, snow_treshold)
    valided_area_snow_modeld(f"{a_date} (with SCF1,4)", observe1_raw, model1_SCF14_raw, snow_treshold)

    valided_area_snow_modeld_return_asc_file(f"{a_date}_without_SCF", observe1_raw, model1_raw, snow_treshold, head_rofental_asc_files)
    valided_area_snow_modeld_return_asc_file(f"{a_date}_with_SCF1,2", observe1_raw, model1_SCF12_raw, snow_treshold, head_rofental_asc_files)
    valided_area_snow_modeld_return_asc_file(f"{a_date}_with_SCF1,4", observe1_raw, model1_SCF14_raw, snow_treshold, head_rofental_asc_files)
    
def importance_of_snowtreshold_validation_area_import_keyfigures_several_models (a_date, snow_treshold):
    input_file_observ = f"results/{a_date}.asc"
    #input_file_model = f"{a_date}_MODEL.asc"
    #input_file_model_SCF12 = f"{a_date}_MODEL-SCF1,2.asc"
    input_file_model_SCF14 = f"results/{a_date}_MODEL-SCF1,4.asc"

    observe1_raw = create_np_arry_from_ASC_file(5, input_file_observ)
    #model1_raw = create_np_arry_from_ASC_file(5, input_file_model)
    #model1_SCF12_raw = create_np_arry_from_ASC_file(5, input_file_model_SCF12)
    model1_SCF14_raw = create_np_arry_from_ASC_file(5, input_file_model_SCF14)

    acc_lst = list()
    bias_lst = list()
    cis_lst = list()
    
    for e in range(len(snow_treshold)):
        #valided_area_snow_modeld(f"{a_date} (without SCF)", observe1_raw, model1_raw, snow_treshold)
        #valided_area_snow_modeld(f"{a_date} (with SCF1,2)", observe1_raw, model1_SCF12_raw), snow_treshold
        acc, bias, cis = valided_area_snow_modeld(f"{a_date} (with SCF1,4)", observe1_raw, model1_SCF14_raw, snow_treshold[e])
        acc_lst.append(acc)
        bias_lst.append(bias)
        cis_lst.append(cis)
    
    keyfigures_with_colors_and_labels = ((acc_lst, "red", "ACC"), (bias_lst, "green", "BIAS"), (cis_lst, "blue", "CIS"))
    plt.figure(figsize=(7, 3), constrained_layout=True)
    for value, color, name in keyfigures_with_colors_and_labels:
        plt.plot(snow_treshold, value, color=color, label=name)
    
    plt.title(f"Auswirkung der Höhe des Schneehöhenschwellenwertes \nauf die Validierung vom {a_date} mit SCF 1.4")
    plt.xlabel("Schwellenwert der Schneehöhe(m)")
    plt.legend()
    plt.savefig(f"{a_date}_schwellenwert_vergleich.svg",dpi=350)

In [9]:

schnee_lst = [i*0.002 for i in range(31)]

#importance_of_snowtreshold_validation_area_import_keyfigures_several_models("2020-06-02", schnee_lst)
#importance_of_snowtreshold_validation_area_import_keyfigures_several_models("2020-07-05", schnee_lst)

In [10]:
schnee_schwellenwert = 0.03
# validating data from 2020-04-11
validation_area_import_keyfigures_asc_file("2020-04-11", schnee_schwellenwert)
print("\n\nNEXT DATE: \n")
validation_area_import_keyfigures_asc_file("2020-06-02", schnee_schwellenwert)
print("NEXT DATE: \n")
validation_area_import_keyfigures_asc_file("2020-07-05", schnee_schwellenwert)


Validating at spatial scale of the  snow_cover from 2020-04-11 (without SCF) 
Accuracy:  0.9235713636776597 Bias:  1.0525590754561958 Critical success Index:  0.9224693679122257

Validating at spatial scale of the  snow_cover from 2020-04-11 (with SCF1,2) 
Accuracy:  0.9334923230671391 Bias:  1.0702706425854953 Critical success Index:  0.9327527684767981

Validating at spatial scale of the  snow_cover from 2020-04-11 (with SCF1,4) 
Accuracy:  0.9287635141273735 Bias:  1.076297495289215 Critical success Index:  0.9283442691864822
Creating a .asc file showing the validation of at spatial scale of the  snow_cover from 2020-04-11_without_SCF 
Creating a .asc file showing the validation of at spatial scale of the  snow_cover from 2020-04-11_with_SCF1,2 
Creating a .asc file showing the validation of at spatial scale of the  snow_cover from 2020-04-11_with_SCF1,4 


NEXT DATE: 


Validating at spatial scale of the  snow_cover from 2020-06-02 (without SCF) 
Accuracy:  0.7890758773371154 Bias